In [5]:
import requests
import re
import datetime
import pickle
import random
import json
import time

In [23]:
# CONSTANTS
GECKO_COOKIE = '__cfduid=d0730edfccacd18b391f1ed3f9616683a1594354492; _vwo_uuid_v2=DF1D4D9E4431CC971F8A37D6C0B3F7E4A|df8cf9347bcc2c8a2d1dce7c43337b83; _ga=GA1.2.582244747.1594354493; _gid=GA1.2.1013311566.1594354493; _vwo_ssm=1; _vis_opt_s=1%7C; _vis_opt_test_cookie=1; _vwo_uuid=DF1D4D9E4431CC971F8A37D6C0B3F7E4A; _session_id=04628290facfb93138a3bee62e274424; _ym_uid=1594354494795729336; _ym_d=1594354494; _ym_isad=1; _ym_visorc_57422968=w; cf_clearance=722574bd28033d3fd3f82ae991191e91695f1ce5-1594354516-0-1zca1cc9eaz8b45487z4f9a6923-150; _vwo_ds=3%3Aa_0%2Ct_0%3A0%241594354492%3A89.41788852%3A%3A%3A54_0%2C7_0%2C6_0%2C5_0%2C2_0%3A0; _gat_gtag_UA_49392197_1=1; _vwo_sn=0%3A22'
TARGET_COIN_SLUGS = [
    'BTC',    'ETH',    'XRP',    'BCH',    'BSV',    'LTC',    'BNB',
    'EOS',    'ADA',    'XTZ',    'LINK',    'XLM',    'XMR',    'NEO',
    'DASH',    'ZEC',    'BAT',    'ZRX',    'NANO',    'VET',    'COMP',
    'RLC'
]
start_date_code = "20130428"

# SEED RAND
random.seed(1)

# OTHER
weekly_dates_backup = []
print weekly_dates_backup

[]


In [7]:
def market_cap_crawler(weeks_to_crawl, pickled_weekly_dates):
    for datecode in weeks_to_crawl:
        print "crawling marketcap for "+datecode
        data_tuple = (datecode, get_total_market_cap(datecode))
        pickled_weekly_dates.append(data_tuple)
        pickle.dump(pickled_weekly_dates, open("weekly_marketcap.p", "wb"))
        weekly_dates_backup = pickled_weekly_dates
        time_to_sleep = random.randint(10,15)
        print "\tSleeping "+str(time_to_sleep)+" seconds"
        time.sleep(time_to_sleep)
        

In [29]:
# updates pickled weekly dates by crawling for updated market cap values
# 

def update_pickled_weekly_dates():
    try:
        pickled_weekly_dates = pickle.load( open("weekly_marketcap.p", "rb"))
        weekly_dates = get_weekly_dates()
        weeks_behind = len(weekly_dates) - len(pickled_weekly_dates)
        if (weeks_behind > 0):
            # behind
            print "Spider bootup..."
            weeks_to_crawl = weekly_dates[-weeks_behind:]
            print str(len(weeks_to_crawl))+" behind..."
            market_cap_crawler(weeks_to_crawl, pickled_weekly_dates)
        return pickled_weekly_dates
            
    except (OSError, IOError) as e:
        # create new pickle file if it doesn't exist
        pickle.dump([], open("weekly_marketcap.p", "wb"))
        
update_pickled_weekly_dates()


Spider bootup...
307 behind...
crawling marketcap for 20140824
	url: https://coinmarketcap.com/historical/20140824
	status code:429
	Couldn't get market cap
	Sleeping 33 seconds
	url: https://coinmarketcap.com/historical/20140824
	status code:200
	Market cap found! - 7,158,980,096
	Sleeping 12 seconds
crawling marketcap for 20140831
	url: https://coinmarketcap.com/historical/20140831
	status code:200
	Market cap found! - 6,846,150,144
	Sleeping 10 seconds
crawling marketcap for 20140907
	url: https://coinmarketcap.com/historical/20140907
	status code:200
	Market cap found! - 6,946,979,840
	Sleeping 10 seconds
crawling marketcap for 20140914
	url: https://coinmarketcap.com/historical/20140914
	status code:200
	Market cap found! - 6,962,259,968
	Sleeping 13 seconds
crawling marketcap for 20140921
	url: https://coinmarketcap.com/historical/20140921
	status code:200
	Market cap found! - 5,859,200,000
	Sleeping 11 seconds
crawling marketcap for 20140928
	url: https://coinmarketcap.com/histo

crawling marketcap for 20150719
	url: https://coinmarketcap.com/historical/20150719
	status code:200
	Market cap found! - 4,585,099,776
	Sleeping 15 seconds
crawling marketcap for 20150726
	url: https://coinmarketcap.com/historical/20150726
	status code:200
	Market cap found! - 4,924,290,048
	Sleeping 13 seconds
crawling marketcap for 20150802
	url: https://coinmarketcap.com/historical/20150802
	status code:200
	Market cap found! - 4,758,850,048
	Sleeping 11 seconds
crawling marketcap for 20150809
	url: https://coinmarketcap.com/historical/20150809
	status code:200
	Market cap found! - 4,545,619,968
	Sleeping 15 seconds
crawling marketcap for 20150816
	url: https://coinmarketcap.com/historical/20150816
	status code:200
	Market cap found! - 4,485,210,112
	Sleeping 15 seconds
crawling marketcap for 20150823
	url: https://coinmarketcap.com/historical/20150823
	status code:200
	Market cap found! - 3,978,579,968
	Sleeping 14 seconds
crawling marketcap for 20150830
	url: https://coinmarketca

	status code:200
	Market cap found! - 12,097,200,128
	Sleeping 10 seconds
crawling marketcap for 20160703
	url: https://coinmarketcap.com/historical/20160703
	status code:200
	Market cap found! - 12,482,100,224
	Sleeping 10 seconds
crawling marketcap for 20160710
	url: https://coinmarketcap.com/historical/20160710
	status code:200
	Market cap found! - 12,300,099,584
	Sleeping 10 seconds
crawling marketcap for 20160717
	url: https://coinmarketcap.com/historical/20160717
	status code:429
	Couldn't get market cap
	Sleeping 38 seconds
	url: https://coinmarketcap.com/historical/20160717
	status code:429
	Couldn't get market cap
	Sleeping 72 seconds
	url: https://coinmarketcap.com/historical/20160717
	status code:200
	Market cap found! - 13,114,200,064
	Sleeping 14 seconds
crawling marketcap for 20160724
	url: https://coinmarketcap.com/historical/20160724
	status code:200
	Market cap found! - 12,954,099,712
	Sleeping 12 seconds
crawling marketcap for 20160731
	url: https://coinmarketcap.com/

crawling marketcap for 20170430
	url: https://coinmarketcap.com/historical/20170430
	status code:200
	Market cap found! - 36,934,901,760
	Sleeping 11 seconds
crawling marketcap for 20170507
	url: https://coinmarketcap.com/historical/20170507
	status code:200
	Market cap found! - 48,488,898,560
	Sleeping 13 seconds
crawling marketcap for 20170514
	url: https://coinmarketcap.com/historical/20170514
	status code:200
	Market cap found! - 55,168,598,016
	Sleeping 15 seconds
crawling marketcap for 20170521
	url: https://coinmarketcap.com/historical/20170521
	status code:200
	Market cap found! - 73,963,102,208
	Sleeping 13 seconds
crawling marketcap for 20170528
	url: https://coinmarketcap.com/historical/20170528
	status code:200
	Market cap found! - 72,521,801,728
	Sleeping 14 seconds
crawling marketcap for 20170604
	url: https://coinmarketcap.com/historical/20170604
	status code:200
	Market cap found! - 93,024,903,168
	Sleeping 13 seconds
crawling marketcap for 20170611
	url: https://coinma

crawling marketcap for 20180318
	url: https://coinmarketcap.com/historical/20180318
	status code:200
	Market cap found! - 318,403,379,200
	Sleeping 15 seconds
crawling marketcap for 20180325
	url: https://coinmarketcap.com/historical/20180325
	status code:200
	Market cap found! - 328,952,676,352
	Sleeping 10 seconds
crawling marketcap for 20180401
	url: https://coinmarketcap.com/historical/20180401
	status code:200
	Market cap found! - 255,105,548,288
	Sleeping 15 seconds
crawling marketcap for 20180408
	url: https://coinmarketcap.com/historical/20180408
	status code:200
	Market cap found! - 266,446,176,256
	Sleeping 10 seconds
crawling marketcap for 20180415
	url: https://coinmarketcap.com/historical/20180415
	status code:200
	Market cap found! - 336,830,660,608
	Sleeping 11 seconds
crawling marketcap for 20180422
	url: https://coinmarketcap.com/historical/20180422
	status code:200
	Market cap found! - 387,669,164,032
	Sleeping 15 seconds
crawling marketcap for 20180429
	url: https://

crawling marketcap for 20190224
	url: https://coinmarketcap.com/historical/20190224
	status code:200
	Market cap found! - 127,409,677,644
	Sleeping 13 seconds
crawling marketcap for 20190303
	url: https://coinmarketcap.com/historical/20190303
	status code:200
	Market cap found! - 129,547,564,672
	Sleeping 15 seconds
crawling marketcap for 20190310
	url: https://coinmarketcap.com/historical/20190310
	status code:200
	Market cap found! - 134,853,788,747
	Sleeping 15 seconds
crawling marketcap for 20190317
	url: https://coinmarketcap.com/historical/20190317
	status code:200
	Market cap found! - 139,704,530,506
	Sleeping 11 seconds
crawling marketcap for 20190324
	url: https://coinmarketcap.com/historical/20190324
	status code:200
	Market cap found! - 140,267,256,987
	Sleeping 13 seconds
crawling marketcap for 20190331
	url: https://coinmarketcap.com/historical/20190331
	status code:200
	Market cap found! - 144,326,758,805
	Sleeping 10 seconds
crawling marketcap for 20190407
	url: https://

	status code:200
	Market cap found! - 288,032,593,680
	Sleeping 14 seconds
crawling marketcap for 20200223
	url: https://coinmarketcap.com/historical/20200223
	status code:200
	Market cap found! - 289,022,567,872
	Sleeping 12 seconds
crawling marketcap for 20200301
	url: https://coinmarketcap.com/historical/20200301
	status code:200
	Market cap found! - 243,559,652,310
	Sleeping 10 seconds
crawling marketcap for 20200308
	url: https://coinmarketcap.com/historical/20200308
	status code:200
	Market cap found! - 231,145,558,855
	Sleeping 11 seconds
crawling marketcap for 20200315
	url: https://coinmarketcap.com/historical/20200315
	status code:429
	Couldn't get market cap
	Sleeping 22 seconds
	url: https://coinmarketcap.com/historical/20200315
	status code:429
	Couldn't get market cap
	Sleeping 78 seconds
	url: https://coinmarketcap.com/historical/20200315
	status code:200
	Market cap found! - 152,899,471,730
	Sleeping 15 seconds
crawling marketcap for 20200322
	url: https://coinmarketcap

NameError: global name 'data' is not defined

In [9]:
def convert_date_to_datecode(date):
    month = "0"+str(date.month) if date.month < 10 else date.month
    day = "0"+str(date.day) if date.day < 10 else date.day
    datecode = str(date.year)+str(month)+str(day)
    assert(len(datecode) == 8)
    return datecode

In [10]:
# Calculate date objects (every 7 days)
# returns list of dates

def get_weekly_dates():
    year = int(start_date_code[0:4])
    month = int(start_date_code[4:6])
    day = int(start_date_code[6:8])
    
    start_date = datetime.date(year, month, day)
    today = datetime.date.today()
    days_diff_datetime = today - start_date
    days_diff = days_diff_datetime.days

    return [convert_date_to_datecode(start_date + datetime.timedelta(days=i)) for i in range(0, days_diff, 7)]

In [20]:
# Pulls the market cap for that week
# date_code {string} - datecode for that week in format YYYYMMDD
def get_total_market_cap(date_code, retries=0):
    url = "https://coinmarketcap.com/historical/"+date_code
    print "\turl: "+url
    r = requests.get(url)
    print "\tstatus code:"+str(r.status_code)
    html = r.text
    total_market_cap = re.findall('Total Market Cap: \$([\d,]+)<\/strong', html)
    try:
        print "\tMarket cap found! - "+total_market_cap[0]
        return total_market_cap[0]
    except IndexError:
        print "\tCouldn't get market cap"
        if (r.status_code >= 400):
            retries+=1
            sleeping = retries*random.randint(20,40)
            print "\tSleeping "+str(sleeping)+" seconds"
            time.sleep(sleeping)
            requests.get("https://coinmarketcap.com/currencies/bitcoin/")
            time.sleep(5)
            if (r.status_code == 404 and retries == 3):
                return None
            return get_total_market_cap(date_code, retries)
        return None


In [19]:
def get_coin_historical_data(coin_path, coin_slug, cookie):
    historical_url = get_historical_url(coin_path)
    headers = {
        'authority': 'www.coingecko.com',
        'method': 'GET',
        'path':'/en/coins/bitcoin/historical_data/usd?end_date=2020-07-10&start_date=2008-07-03',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        'cookie': cookie,
        'dnt':'1',
        'sec-fetch-dest':'document',
        'sec-fetch-mode':'navigate',
        'sec-fetch-site':'same-origin',
        'sec-fetch-user':'?1',
        'upgrade-insecure-requests':'1',
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    }
    r = requests.get(historical_url, headers=headers, stream=True)
    html = r.text
    history = re.findall(r'class="font-semibold text-center">(\d{4}-\d{2}-\d{2})<\/th>\s<td class="text-center">\s\$[\d,]+.\d{2}\s<\/td>\s<td class="text-center">\s\$([\d,]+.\d{2})\s<\/td>\s<\/tr>', html)
    return history[0::7] # get every 7th element


In [15]:
def get_historical_url(coin_path):
    current_date = datetime.today().strftime('%Y-%m-%d')
    return "https://www.coingecko.com/en/coins/"+coin_path+"/historical_data/usd?end_date="+current_date+"&start_date=2008-07-03#panel"

In [4]:
def pull_coin_list():
    r = requests.get("https://www.coingecko.com/en");
    html = r.text
    coin_list = re.findall(r'font-bold\"\shref="\/en\/coins\/(.*)\">\s(.*)\s<', html)
    assert(len(coin_list) == 100)
    return coin_list

In [36]:
# MAIN
coin_list = pull_coin_list()
get_coin_historical_data('bitcoin','BTC', GECKO_COOKIE)

NameError: name 'pull_coin_list' is not defined